# LoRa Transmission Data Analysis

This notebook analyses lora transmission data collected at the Ol Pejeta Conservancy between the 19th and 20th of September. The aim of the data collection exercise was to determine transmission range of the lora enabled sensors within the conservancy and to test the deployment of river level monitoring sensors along the Ewaso Nyiro River.

There are two LoRa gateways mounted at Ol Pejeta House. A Kerlink Gateway at approximately 16m and a Lorix One gateway at approximately 13m.

We deployed three mdot based lora devices at various locations within the conservancy. We used the Maxbotix ultrasonic range sensor with a range of 6m to determine the depth of the Ewaso Nyiro.

The devices and gateway are registered on the Things Network and we use a custom script based on TTN's python SDK to retrive the data and write it to an Influx Database. We store radio propagation parameters including RSSI, SNR and datarate. We vary the datarate between DR0-DR6 and transmit every minute.

![map|512x397,60%](img/opc-sensor-points.jpg)


In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas import DataFrame, Series
from pandas.io.json import json_normalize

from influxdb import InfluxDBClient
from datetime import datetime, timedelta, timezone

from geopy.distance import geodesic

from tabulate import tabulate

client = InfluxDBClient(host='localhost', port=8086)
client.switch_database('ol_pejeta_mdot_range_test')


In [9]:
result = client.query('select * from "ol_pejeta_mdot_range_test" where time > now() - 180d')

InfluxDBClientError: database not found: ol_pejeta_mdot_range_test

In [3]:
# Compute distances from points to gateways

kerlink_loc = (0.026884535, 36.903187)
lorix_loc = (0.026812032, 36.9032)

mdot_01_p1_loc = (0.025448333, 36.903027)
mdot_02_p1_loc = (0.022585, 36.905968)
mdot_01_p2_loc = (0.0302132, 36.90529)
mdot_02_p2_loc = (-0.0042156363, 36.963684)
mdot_03_p2_loc = (0.026764926, 36.914623)

sensors = ['mdot-01-P1', 
           'mdot-02-P1', 
           'mdot-01-P2',
           'mdot_02_P2',
           'mdot_03_P3']
sensor_locs = [mdot_01_p1_loc, 
               mdot_02_p1_loc,
               mdot_01_p2_loc,
               mdot_02_p2_loc,
               mdot_03_p2_loc]
kerlink_dist = []

for sl in sensor_locs:
    kerlink_dist.append([sensors[sensor_locs.index(sl)],
                         geodesic(kerlink_loc, sl).km])
    
print(tabulate(kerlink_dist, ['Sensor', 'Distance (km)']))

lorix_dist = []

for sl in sensor_locs:
    lorix_dist.append([sensors[sensor_locs.index(sl)],
                         geodesic(lorix_loc, sl).km])
    
print(tabulate(lorix_dist, ['Sensor', 'Distance (km)']))

Sensor        Distance (km)
----------  ---------------
mdot-01-P1         0.159803
mdot-02-P1         0.567329
mdot-01-P2         0.436207
mdot_02_P2         7.5617
mdot_03_P3         1.27312
Sensor        Distance (km)
----------  ---------------
mdot-01-P1         0.152015
mdot-02-P1         0.55983
mdot-01-P2         0.44223
mdot_02_P2         7.55677
mdot_03_P3         1.27161


In [4]:
# Get the entries from the last hour
result = client.query('select * from "Range Test"')

result_list = list(result.get_points())

# turn to pandas dataframe
df = pd.DataFrame(result_list)


# make time a datetime object
df[['time']] = df[['time']].apply(pd.to_datetime)



InfluxDBClientError: database not found: ol_pejeta_mdot_range_test

## Position 1
Poistion 1 locations were deployed between 11.30am and 1.30pm on the 19th of September.

In [ ]:
start_time = datetime(2019, 9, 19, 8, 30, tzinfo=timezone.utc) # time in utc
stop_time = datetime(2019, 9, 19, 10, 30, tzinfo=timezone.utc) # time in utc

sensors = df.groupby('sensor')

sensor_labels = ['mdot-one', 'mdot-two']

for sensor_label in sensor_labels:
    sensor = sensors.get_group(sensor_label)

    sensor_p1 = sensor[(sensor['time']>=start_time) &
                        (sensor['time']<=stop_time)]

    for label, grp in sensor_p1.groupby('gateway'):
        print(sensor_label, label, np.mean(grp.rssi), np.std(grp.rssi))

## Position 2

In [ ]:
start_time = datetime(2019, 9, 19, 15, 0, tzinfo=timezone.utc) # time in utc
stop_time = datetime(2019, 9, 20, 11, 0, tzinfo=timezone.utc) # time in utc

sensors = df.groupby('sensor')

sensor_labels = ['mdot-one', 'mdot-two', 'mdot-03']

for sensor_label in sensor_labels:
    sensor = sensors.get_group(sensor_label)

    sensor_p2 = sensor[(sensor['time']>=start_time) &
                        (sensor['time']<=stop_time)]

    for label, grp in sensor_p2.groupby('gateway'):
        print(sensor_label, label, np.mean(grp.rssi), np.std(grp.rssi))